# RogueSky

author: camen  
last run: 2019-11-10

In [1]:
import contextlib
import json
import logging
import os

import requests_mock

from rogue_sky import darksky, postgres_utilities

TEST_DATABASE_URL = os.environ["TEST_DATABASE_URL"]
DARKSKY_API_KEY = "test_api_key"

COORDINATES = (47.6062, -122.3321)  # seattle
DARKSKY_URL = f"https://api.darksky.net/forecast/{DARKSKY_API_KEY}/{COORDINATES[0]},{COORDINATES[1]}"

logging.basicConfig(level=logging.INFO)

## Table of Contents

1. [Set Up](#Set-Up)
2. [Introduction](#Introduction)
3. [Working with DarkSky](#Working-with-DarkSky)
    1. [Using the main entry point](#Using-the-main-entry-point...)
    2. [Going through the steps](#Going-through-the-steps...)
        1. [Check the database](#From-the-database)
        2. [From DarkSky](#From-DarkSky)
        3. [To the database](#To-the-database)
        4. [Serialize the output](#Serialize-the-output)
    3. [Understanding the data](#Understanding-the-data)

## Set Up

In [2]:
# set up the test database
with postgres_utilities.get_cursor(pg_url=TEST_DATABASE_URL) as cursor:
    cursor.execute("DROP TABLE IF EXISTS daily_weather_forecast")
    
postgres_utilities.create_weather_table(pg_url=TEST_DATABASE_URL)

In [3]:
# set up the mock for requests (we won't actually hit the DarkSky API)
@contextlib.contextmanager
def mock_darksky_api():
    with requests_mock.Mocker() as mock:
        with open("resources/darksky_response.json") as json_file:
            data = json.load(json_file)
        mock.get(DARKSKY_URL, json=data)
        yield

## Introduction

Predict star visibility from DarkSky's daily weather forecast. In this notebook, we will query DarkSky for a daily weather forecast, show some information about that forecast, and predict the star visibility from it.

## Working with DarkSky

### Using the main entry point...

First, lets see it all work together... The main entry point into the darksky module is the `get_weather_forecast` method. This method:
    1. Checks the local database to see if a DarkSky query for the same lat-lon coordinates has already been made today, if not it returns that.
    2. If there no query has been made today, then it hits the DarkSky API for the daily weather forecast, and persists that to the local database.
    3. Serializes the daily weather forecast into a JSON-parseable response that can be sent my the RogueSky API to the RogueSky frontend.

The final output is of this form: 
```
{
    0: {
        "latitude": 42.3601,
        "longitude": -71.0589,
        "queried_date_utc": "2019-01-01",
        "weather_date_utc": "2019-11-10",
        "weather_json": weather_json,
    },
    1: {
        "latitude": 42.3601,
        "longitude": -71.0589,
        "queried_date_utc": "2019-01-01",
        "weather_date_utc": "2019-11-11",
        "weather_json": weather_json,
    },
    ...
}
```

In [4]:
with mock_darksky_api():
    backend_response = darksky.get_weather_forecast(
        latitude=COORDINATES[0], 
        longitude=COORDINATES[1],
        api_key=DARKSKY_API_KEY,
        database_url=TEST_DATABASE_URL
    )

backend_response["0"]

INFO:rogue_sky.darksky:Getting daily weather forecast for (47.6062, -122.3321) on 2019-11-14
INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-11-14): Checking database...
INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-11-14): Not found in database...
INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-11-14): Requesting from DarkSky with parameters {'query_parameters': {'exclude': ['alerts', 'flags', 'hourly', 'minutely', 'offset', 'currently']}}
INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-11-14): Persisting to database...
INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-11-14): Serializing to API output...


{'latitude': 47.6062,
 'longitude': -122.3321,
 'queried_date_utc': '2019-11-14',
 'weather_date_utc': '2019-11-10',
 'weather_json': {'cloud_cover_pct': 0.93,
  'dew_point_f': 48.57,
  'humidity_pct': 0.94,
  'moon_phase_pct': 0.46,
  'ozone': 262.2,
  'precip_intensity_avg_in_hr': 0.0004,
  'precip_intensity_max_in_hr': 0.0024,
  'precip_probability': 0.34,
  'precip_type': 'rain',
  'pressure': 1026.1,
  'sunrise_time_utc': '2019-11-10T15:09:00+00:00',
  'sunset_time_utc': '2019-11-11T00:41:00+00:00',
  'temperature_max_f': 53.25,
  'temperature_max_time_utc': '2019-11-10T21:56:00+00:00',
  'temperature_min_f': 44.73,
  'temperature_min_time_utc': '2019-11-11T08:00:00+00:00',
  'uv_index': 1,
  'visibility_mi': 5.174,
  'weather_date_utc': '2019-11-10',
  'wind_gust_mph': 11.25,
  'wind_speed_mph': 3.93}}

### Going through the steps...

1. Check the database (`darksky._from_database`)
    1. If found, serialize (`darksky._serialize`) and return
1. If not found, request from darksky (`darksky._from_darksky`)
2. Persist to database (`darksky._to_database`)
3. Serialize (`darksky._serialize`) and return

#### From the database

```
[
    {
        latitude: 42.3601,
        longitude: -71.0589,
        queried_date_utc: "2019-01-01",
        weather_date_utc: "2019-01-01",
        weather_json: JSON({
            ...
        })
    },
    ...
]
```

In [5]:
# Finding the above weather forecast
from_database_response = darksky._from_database(
    latitude=COORDINATES[0], 
    longitude=COORDINATES[1], 
    queried_date_utc=backend_response["0"]["queried_date_utc"], 
    database_url=TEST_DATABASE_URL
)

from_database_response[0]  # type signature must match the output from `_from_darksky` below

INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-11-14): Checking database...
INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-11-14): Found in database!


{'latitude': 47.6062,
 'longitude': -122.3321,
 'queried_date_utc': '2019-11-14',
 'weather_date_utc': '2019-11-10',
 'weather_json': '{"cloud_cover_pct": 0.93, "dew_point_f": 48.57, "humidity_pct": 0.94, "moon_phase_pct": 0.46, "ozone": 262.2, "precip_intensity_avg_in_hr": 0.0004, "precip_intensity_max_in_hr": 0.0024, "precip_probability": 0.34, "precip_type": "rain", "pressure": 1026.1, "sunrise_time_utc": "2019-11-10T15:09:00+00:00", "sunset_time_utc": "2019-11-11T00:41:00+00:00", "temperature_max_f": 53.25, "temperature_max_time_utc": "2019-11-10T21:56:00+00:00", "temperature_min_f": 44.73, "temperature_min_time_utc": "2019-11-11T08:00:00+00:00", "uv_index": 1, "visibility_mi": 5.174, "weather_date_utc": "2019-11-10", "wind_gust_mph": 11.25, "wind_speed_mph": 3.93}'}

In [6]:
# Now, let's remove the data persisted from our `backend.get_weather_forecast()` 
# call so that we can re-upload it (there is a unique constraint on the 
# latitude, longitude, and query date).

with postgres_utilities.get_cursor(pg_url=TEST_DATABASE_URL) as cursor:
    cursor.execute("TRUNCATE TABLE daily_weather_forecast")

In [7]:
# Not finding daily weather
# We shouldn't find the forecast now since we just truncated the table.
from_database_response = darksky._from_database(
    latitude=COORDINATES[0], 
    longitude=COORDINATES[1], 
    queried_date_utc=backend_response["0"]["queried_date_utc"], 
    database_url=TEST_DATABASE_URL
)

from_database_response

INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-11-14): Checking database...
INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-11-14): Not found in database...


[]

#### From DarkSky

```
[
    {
        latitude: 42.3601,
        longitude: -71.0589,
        queried_date_utc: "2019-01-01",
        weather_date_utc: "2019-01-01",
        weather_json: JSON({
            ...
        })
    },
    ...
]
```

In [8]:
# If could not find in the database we would...
# query DarkSky and parse...
with mock_darksky_api():
    darksky_response = darksky._from_darksky(
        latitude=COORDINATES[0],
        longitude=COORDINATES[1],
        queried_date_utc=backend_response["0"]["queried_date_utc"],
        api_key=DARKSKY_API_KEY,
    )

darksky_response[0]

INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-11-14): Requesting from DarkSky with parameters {'query_parameters': {'exclude': ['alerts', 'flags', 'hourly', 'minutely', 'offset', 'currently']}}


{'latitude': 47.6062,
 'longitude': -122.3321,
 'queried_date_utc': '2019-11-14',
 'weather_date_utc': '2019-11-10',
 'weather_json': '{"cloud_cover_pct": 0.93, "dew_point_f": 48.57, "humidity_pct": 0.94, "moon_phase_pct": 0.46, "ozone": 262.2, "precip_intensity_avg_in_hr": 0.0004, "precip_intensity_max_in_hr": 0.0024, "precip_probability": 0.34, "precip_type": "rain", "pressure": 1026.1, "sunrise_time_utc": "2019-11-10T15:09:00+00:00", "sunset_time_utc": "2019-11-11T00:41:00+00:00", "temperature_max_f": 53.25, "temperature_max_time_utc": "2019-11-10T21:56:00+00:00", "temperature_min_f": 44.73, "temperature_min_time_utc": "2019-11-11T08:00:00+00:00", "uv_index": 1, "visibility_mi": 5.174, "weather_date_utc": "2019-11-10", "wind_gust_mph": 11.25, "wind_speed_mph": 3.93}'}

#### To the database

In [9]:
# if we could not find in the database we would...
# query DarkSky, parse it, and persist to the database...

darksky._to_database(response=darksky_response, database_url=TEST_DATABASE_URL)

INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-11-14): Persisting to database...


#### Serialize the output

```
{
    0: {
        "latitude": 42.3601,
        "longitude": -71.0589,
        "queried_date_utc": "2019-11-10",
        "weather_date_utc": "2019-11-10",
        "weather_json": weather_json,
    },
    1: {
        "latitude": 42.3601,
        "longitude": -71.0589,
        "queried_date_utc": "2019-01-01",
        "weather_date_utc": "2019-11-11",
        "weather_json": weather_json,
    },
    ...
}
```

In [10]:
serialized_daily_forecast = darksky._serialize(response=darksky_response)

serialized_daily_forecast["0"]

INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-11-14): Serializing to API output...


{'latitude': 47.6062,
 'longitude': -122.3321,
 'queried_date_utc': '2019-11-14',
 'weather_date_utc': '2019-11-10',
 'weather_json': {'cloud_cover_pct': 0.93,
  'dew_point_f': 48.57,
  'humidity_pct': 0.94,
  'moon_phase_pct': 0.46,
  'ozone': 262.2,
  'precip_intensity_avg_in_hr': 0.0004,
  'precip_intensity_max_in_hr': 0.0024,
  'precip_probability': 0.34,
  'precip_type': 'rain',
  'pressure': 1026.1,
  'sunrise_time_utc': '2019-11-10T15:09:00+00:00',
  'sunset_time_utc': '2019-11-11T00:41:00+00:00',
  'temperature_max_f': 53.25,
  'temperature_max_time_utc': '2019-11-10T21:56:00+00:00',
  'temperature_min_f': 44.73,
  'temperature_min_time_utc': '2019-11-11T08:00:00+00:00',
  'uv_index': 1,
  'visibility_mi': 5.174,
  'weather_date_utc': '2019-11-10',
  'wind_gust_mph': 11.25,
  'wind_speed_mph': 3.93}}

### Understanding the data

```
{
    'latitude': 47.6062,
    'longitude': -122.3321,
    'timezone': 'America/Los_Angeles',
    'currently': dict  current weather,
    'minutely': {
        'icon': 'cloudy',
        'data': [dict]  forecast by minute for 60 minutes
    },
    'hourly': {
        'summary': 'cloudy',
        'icon': 'cloudy',
        'data': [dict]  forecast by hour for 48 hours
    },
    'daily': {
        'summary': 'cloudy',
        'icon': 'cloudy',
        'data': [dict]  forecast by day for 8 days, including today
    }
    
```

In [11]:
# the response from DarkSky

with open("resources/darksky_response.json") as json_file:
    data = json.load(json_file)

In [12]:
data.keys()

dict_keys(['latitude', 'longitude', 'timezone', 'currently', 'minutely', 'hourly', 'daily', 'flags', 'offset'])

In [13]:
data["daily"].keys()

dict_keys(['summary', 'icon', 'data'])

In [14]:
type(data["daily"]["data"])

list

In [15]:
list(data["daily"]["data"][0].keys())

['time',
 'summary',
 'icon',
 'sunriseTime',
 'sunsetTime',
 'moonPhase',
 'precipIntensity',
 'precipIntensityMax',
 'precipIntensityMaxTime',
 'precipProbability',
 'precipType',
 'temperatureHigh',
 'temperatureHighTime',
 'temperatureLow',
 'temperatureLowTime',
 'apparentTemperatureHigh',
 'apparentTemperatureHighTime',
 'apparentTemperatureLow',
 'apparentTemperatureLowTime',
 'dewPoint',
 'humidity',
 'pressure',
 'windSpeed',
 'windGust',
 'windGustTime',
 'windBearing',
 'cloudCover',
 'uvIndex',
 'uvIndexTime',
 'visibility',
 'ozone',
 'temperatureMin',
 'temperatureMinTime',
 'temperatureMax',
 'temperatureMaxTime',
 'apparentTemperatureMin',
 'apparentTemperatureMinTime',
 'apparentTemperatureMax',
 'apparentTemperatureMaxTime']

In [16]:
# times are in UNIX time
# `time` is in local time, but all other times are in UTC (e.g. `sunriseTime`)

data["daily"]["data"][0]

{'time': 1573372800,
 'summary': 'Foggy in the morning.',
 'icon': 'fog',
 'sunriseTime': 1573398540,
 'sunsetTime': 1573432860,
 'moonPhase': 0.46,
 'precipIntensity': 0.0004,
 'precipIntensityMax': 0.0024,
 'precipIntensityMaxTime': 1573448400,
 'precipProbability': 0.34,
 'precipType': 'rain',
 'temperatureHigh': 53.25,
 'temperatureHighTime': 1573422960,
 'temperatureLow': 40.82,
 'temperatureLowTime': 1573481400,
 'apparentTemperatureHigh': 52.75,
 'apparentTemperatureHighTime': 1573422960,
 'apparentTemperatureLow': 38.74,
 'apparentTemperatureLowTime': 1573481520,
 'dewPoint': 48.57,
 'humidity': 0.94,
 'pressure': 1026.1,
 'windSpeed': 3.93,
 'windGust': 11.25,
 'windGustTime': 1573434060,
 'windBearing': 353,
 'cloudCover': 0.93,
 'uvIndex': 1,
 'uvIndexTime': 1573415700,
 'visibility': 5.174,
 'ozone': 262.2,
 'temperatureMin': 44.73,
 'temperatureMinTime': 1573459200,
 'temperatureMax': 53.25,
 'temperatureMaxTime': 1573422960,
 'apparentTemperatureMin': 42.78,
 'apparentTem

In [17]:
from rogue_sky import stars

In [20]:
with open("/Users/camen/Documents/rogue-sky/backend/tests/resources/test_serialized_weather_forecast.json", "w") as outfile:
    json.dump(serialized_daily_forecast, outfile)